In [1]:
import numpy as np
import torch
from transformers import BertTokenizer,BertForSequenceClassification,Trainer,TrainingArguments
from sklearn.model_selection import train_test_split

In [2]:
texts = ["I love transformers", "This movie was terrible", "Amazing story!", "I hate this phone"]
labels = [1, 0, 1, 0]  # 1 = positive, 0 = negative


train_texts, val_texts, train_labels, val_labels = train_test_split(texts, labels, test_size=0.2)

# Tokenization
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
train_enc = tokenizer(train_texts, padding=True, truncation=True,return_tensors="pt")
val_enc = tokenizer(val_texts, padding=True, truncation=True, return_tensors="pt")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [3]:
from torch.utils.data import Dataset
class SentimenetDataset(Dataset):
    def __init__(self,encodings,labels):
        self.encodings = encodings
        self.labels = torch.tensor(labels)
    def __getitem__(self, index):
        item = {k: v[index] for k,v in self.encodings.items()}
        item['labels'] = self.labels[index]
        return item
    def __len__(self):
        return len(self.labels)
train_datset = SentimenetDataset(train_enc,train_labels)
val_dataset = SentimenetDataset(val_enc,val_labels)

In [4]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
training_args = TrainingArguments(
    output_dir='./sample_data',
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    eval_strategy="epoch",
    logging_dir='./logs',
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_datset,
    eval_dataset=val_dataset,
)

trainer.train()


Epoch,Training Loss,Validation Loss
1,No log,0.981712
2,No log,1.187040
3,No log,1.256428


TrainOutput(global_step=3, training_loss=0.41944026947021484, metrics={'train_runtime': 12.0308, 'train_samples_per_second': 0.748, 'train_steps_per_second': 0.249, 'total_flos': 27749994120.0, 'train_loss': 0.41944026947021484, 'epoch': 3.0})

In [6]:
test_input = tokenizer("This phone is fantastic!", return_tensors="pt", padding=True, truncation=True)
output = model(**test_input)
probs = torch.softmax(output.logits, dim=1)
pred = torch.argmax(probs, dim=1)
print("Predicted Sentiment:", "Positive" if pred == 1 else "Negative")


Predicted Sentiment: Positive
